In [1]:
!pip install adversarial-robustness-toolbox

You should consider upgrading via the 'C:\Users\anssi\Desktop\ai\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
from art.estimators.object_detection.pytorch_detection_transformer import PyTorchDetectionTransformer
from art.attacks.evasion.adversarial_patch.adversarial_patch_pytorch import AdversarialPatchPyTorch
from torchvision.transforms import transforms
import PIL.Image
import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt
import requests
import pandas as pd

import fiftyone as fo
import fiftyone.zoo as foz

C:\Users\anssi\Desktop\ai\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(fo.list_datasets())

#dataset = foz.load_zoo_dataset(
 #   "coco-resized-to-800",
  #  split="validation",
   # dataset_name="dpatched-images",
#)

name="patched-321212121"
dataset_dir="./coco_resized/validation"
dataset_type = fo.types.COCODetectionDataset
dataset = fo.Dataset.from_dir(dataset_dir, dataset_type, name=name)
dataset.persistent = False
predictions_view = dataset.take(5000, seed=51)

['coco-123', 'coco-1234', 'coco-2017-validation-50', 'coco-800', 'coco-most_original', 'coco-original', 'coco-patched-1', 'coco-patched-1234', 'coco-patched-12345', 'coco-patched-test1', 'coco-patched-test12', 'coco-re', 'coco-res', 'coco-resized', 'coco-resized-to-800', 'coco-resized-to-800-2', 'coco-resizer', 'coco-to-dpatch', 'coco-to-dpatch-2', 'coco-to-dpatch-3', 'coco-to-dpatch-4', 'coco-to-dpatch-5', 'coco-to-dpatch-6', 'coco-to-resized_testing', 'coco-to-resized_testing-2', 'coco-to-resized_testing-3', 'coco-to-resized_testing-4', 'coco-to-resized_testing-5', 'coco_224', 'evaluate-detections-tutorial', 'patched', 'patched-1', 'patched-2', 'patched-3', 'patched-31233445365', 'patched-312334453655', 'patched-3123345', 'patched-321', 'resize', 'test']
 100% |███████████████| 5000/5000 [18.7s elapsed, 0s remaining, 274.1 samples/s]      


In [4]:
COCO_CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]


In [5]:
def filter_boxes(predictions, conf_thresh):
    dictionary = {}

    boxes_list = []
    scores_list = []
    labels_list = []

    for i in range(len(predictions[0]["boxes"])):
        score = predictions[0]["scores"][i]
        if score >= conf_thresh:
            boxes_list.append(predictions[0]["boxes"][i])
            scores_list.append(predictions[0]["scores"][[i]])
            labels_list.append(predictions[0]["labels"][[i]])

    dictionary["boxes"] = np.vstack(boxes_list)
    dictionary["scores"] = np.hstack(scores_list)
    dictionary["labels"] = np.hstack(labels_list)

    y = [dictionary]

    return y

In [6]:
def plot_image_with_boxes(img, boxes, pred_cls, title):
    text_size = 2
    text_th = 2
    rect_th = 2

    for i in range(len(boxes)):
        cv2.rectangle(img, (int(boxes[i][0][0]), int(boxes[i][0][1])), (int(boxes[i][1][0]), int(boxes[i][1][1])),
                      color=(0, 255, 0), thickness=rect_th)
        cv2.putText(img, pred_cls[i], (int(boxes[i][0][0]), int(boxes[i][0][1])), cv2.FONT_HERSHEY_SIMPLEX, text_size,
                    (0, 255, 0), thickness=text_th)
    plt.figure()
    plt.axis("off")
    plt.title(title)
    plt.imshow(img)

In [7]:
def extract_predictions(predictions_, conf_thresh):
    predictions_class = [COCO_CLASSES[i] for i in list(predictions_["labels"])]
    if len(predictions_class) < 1:
        return [], [], []
        
    predictions_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(predictions_["boxes"])]
    predictions_score = list(predictions_["scores"])

    threshold = conf_thresh
    predictions_t = [predictions_score.index(x) for x in predictions_score if x > threshold]
    if len(predictions_t) > 0:
        predictions_t = predictions_t
    else:
        return [], [], []
        
    predictions_boxes = [predictions_boxes[i] for i in predictions_t]
    predictions_class = [predictions_class[i] for i in predictions_t]
    predictions_scores = [predictions_score[i] for i in predictions_t]
    return predictions_class, predictions_boxes, predictions_scores

In [8]:
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
NUMBER_CHANNELS = 3
INPUT_SHAPE = (NUMBER_CHANNELS, 800, 800)

transform = transforms.Compose([
        transforms.Resize([INPUT_SHAPE[1], INPUT_SHAPE[2]], interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.ToTensor()
    ])

In [9]:
detector = PyTorchDetectionTransformer(channels_first=True, preprocessing=(MEAN, STD), input_shape=INPUT_SHAPE, clip_values=(0,1))

Using cache found in C:\Users\anssi/.cache\torch\hub\facebookresearch_detr_main
C:\Users\anssi\Desktop\ai\env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\anssi\Desktop\ai\env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
import re

def extract_numbers(filename):
   numbers = re.findall(r'\d+', filename)
   return ''.join(numbers)

In [17]:
import os
coco_images = []
image_ids = []
image_sizes = []
unsized_images = []
for sample in predictions_view:
    image_id = os.path.basename(sample.filepath)
    image_id = image_id.lstrip('0')
    image_id = image_id.replace('.jpg', '')
    image_ids.append(image_id)
    im = PIL.Image.open(sample.filepath)
    im = im.convert('RGB')
    image_sizes.append(im.size)
    #print(im.dtype) # Print the data type of the image
    #print(f"shape before: {im.shape}")
    im = transform(im).numpy()
    #print(f"shape after: {im.shape}")
    if im.shape != (3, 800, 800):
        print("size doesn't match")
        print(im.shape)

    coco_images.append(im)
coco_images = np.array(coco_images)



MemoryError: Unable to allocate 35.8 GiB for an array with shape (5000, 3, 800, 800) and data type float32

In [ ]:
#pullaa nämä fiftyonesta:
dets = detector.predict(coco_images[:20]) # ttämä pakko rajottaa ja keksiä joku syy dippaan sille...
filtered_dets = [filter_boxes([t], 0.8)[0] for t in dets]

x = coco_images[:-1]
targets = [filtered_dets[-1] for i in range(len(x))]

rotation_max=0.0
scale_min=0.5
scale_max=1
distortion_scale_max=0.0
learning_rate=1.99
max_iter=400
batch_size=16
patch_shape=(3, 200, 200)
patch_location=(100,100)
patch_type="square"
optimizer="adam"

ap = AdversarialPatchPyTorch(estimator=detector, rotation_max=rotation_max, 
                      scale_min=scale_min, scale_max=scale_max, distortion_scale_max=distortion_scale_max,
                      learning_rate=learning_rate, max_iter=max_iter, batch_size=batch_size, patch_location=patch_location,
                      patch_shape=patch_shape, patch_type=patch_type, verbose=True, targeted=False)

patch, patch_mask = ap.generate(x=x[[0]], y=filtered_dets[:1])

plt.axis("off")
plt.imshow(((patch) * patch_mask).transpose(1,2,0))
plt.show()

In [ ]:
# TÄHÄN NYT TUON COCO_IMAGESIN TILALLE FIFTYONESTA NE KUVAT
# Nyt kun saadaan fiftyonesta kuvat
# pitää tallentaa nämä patchatyt kuvat levylle jonnekin...
import cv2
import imageio
#for image in coco_images:
length = len(coco_images)
patched_images = ap.apply_patch(coco_images[:length], scale=0.4)
i=0
for image in patched_images:
    #print(image)
    image = (image * 255).astype('uint8')
    image = np.transpose(image, (1, 2, 0))
    img = PIL.Image.fromarray(image)
    img.save(f'E:\\coco_patched\\data\\{image_ids[i]}.jpg')
    print("image saved")
    # this still needs to be resized into the original output and needs the correct id attached...
    i+=1
#dets = detector.predict(patched_images)
#for i in range(len(dets)):
 #   preds_orig = extract_predictions(dets[i], 0.8)
  #  plot_image_with_boxes(img=patched_images[i].transpose(1,2,0).copy(), boxes=preds_orig[1], pred_cls=preds_orig[0],
   #                        title="Predictions on image with patch")

